In [264]:
import json
import sqlite3
import requests
import numpy as np
import pandas as pd
from pathlib import Path

DB_PATH = Path("../Data/DB/EXAM.db")

In [265]:
prod_raw = pd.read_csv(
    Path.cwd().parent / "Data" / "Row" / "products_noisy.csv"
)
prod_raw

,ProductID,ProductName,Category,Price
0,101.0,Product_1,Books,NaN
1,102.0,Product_2,NaN,438.65
2,103.0,Product_3,Electronics,-1.00
3,104.0,Product_4,NaN,349.40
4,105.0,Product_5,Home,280.33
...,...,...,...,...
995,1096.0,Product_996,Sports,NaN
996,1097.0,Product_997,Toys,NaN
997,1098.0,NaN,Clothing,333.56
998,1099.0,Product_999,Sports,459.75


In [266]:
prod_raw.isna().sum()

ProductID       87
ProductName     90
Category       117
Price          130
dtype: int64

In [267]:
len(prod_raw["ProductID"].unique())

819

In [268]:
prod_raw

,ProductID,ProductName,Category,Price
0,101.0,Product_1,Books,NaN
1,102.0,Product_2,NaN,438.65
2,103.0,Product_3,Electronics,-1.00
3,104.0,Product_4,NaN,349.40
4,105.0,Product_5,Home,280.33
...,...,...,...,...
995,1096.0,Product_996,Sports,NaN
996,1097.0,Product_997,Toys,NaN
997,1098.0,NaN,Clothing,333.56
998,1099.0,Product_999,Sports,459.75


In [269]:
prod_raw["ProductID"].nunique()

818

In [270]:
prod_raw["ProductID"] = range(101, len(prod_raw) + 101)

In [271]:
prod_raw["ProductID"].nunique()

1000

In [272]:
prod_raw["ProductName"] = [f"Product_{i}" for i in range(1, 1001)]

In [273]:
prod_raw

,ProductID,ProductName,Category,Price
0,101,Product_1,Books,NaN
1,102,Product_2,NaN,438.65
2,103,Product_3,Electronics,-1.00
3,104,Product_4,NaN,349.40
4,105,Product_5,Home,280.33
...,...,...,...,...
995,1096,Product_996,Sports,NaN
996,1097,Product_997,Toys,NaN
997,1098,Product_998,Clothing,333.56
998,1099,Product_999,Sports,459.75


In [274]:
prod_raw["Category"] = prod_raw["Category"].fillna(prod_raw["Category"].mode()[0])

In [275]:
prod_raw

,ProductID,ProductName,Category,Price
0,101,Product_1,Books,NaN
1,102,Product_2,Home,438.65
2,103,Product_3,Electronics,-1.00
3,104,Product_4,Home,349.40
4,105,Product_5,Home,280.33
...,...,...,...,...
995,1096,Product_996,Sports,NaN
996,1097,Product_997,Toys,NaN
997,1098,Product_998,Clothing,333.56
998,1099,Product_999,Sports,459.75


In [276]:
prod_raw["Price"].min()

np.float64(-1.0)

In [277]:
prod_raw["Price"].max()

np.float64(498.43)

In [278]:
prod_raw["Price"] = prod_raw["Price"].mask(prod_raw["Price"] <= 0, prod_raw["Price"].median()).fillna(prod_raw["Price"].median())

In [279]:
prod_raw["Price"].isna().sum()

np.int64(0)

In [280]:
prod_raw

,ProductID,ProductName,Category,Price
0,101,Product_1,Books,225.09
1,102,Product_2,Home,438.65
2,103,Product_3,Electronics,225.09
3,104,Product_4,Home,349.40
4,105,Product_5,Home,280.33
...,...,...,...,...
995,1096,Product_996,Sports,225.09
996,1097,Product_997,Toys,225.09
997,1098,Product_998,Clothing,333.56
998,1099,Product_999,Sports,459.75


In [281]:
prod_raw.isna().sum()

ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64

In [282]:
inv_raw = pd.read_csv(
    Path.cwd().parent / "Data" / "Row" / "inventory_noisy.csv"
)
inv_raw

,InventoryID,ProductID,WarehouseCode,StockLevel
0,-1.0,101.0,WH-A,NaN
1,2.0,NaN,WH-A,20.0
2,3.0,103.0,WH-A,20.0
3,4.0,104.0,WH-A,20.0
4,5.0,105.0,WH-A,20.0
...,...,...,...,...
995,996.0,1096.0,WH-B,135.0
996,997.0,1097.0,NaN,126.0
997,-1.0,1098.0,NaN,63.0
998,999.0,-1.0,WH-A,1.0


In [283]:
inv_raw["ProductID"] = range(101, len(inv_raw) + 101)

In [284]:
inv_raw["InventoryID"] = range(1, len(inv_raw) + 1)

In [285]:
inv_raw["WarehouseCode"] = inv_raw["WarehouseCode"].fillna(inv_raw["WarehouseCode"].mode()[0])

In [286]:
inv_raw["StockLevel"] = inv_raw["StockLevel"].fillna(inv_raw["StockLevel"].median())

In [287]:
inv_raw

,InventoryID,ProductID,WarehouseCode,StockLevel
0,1,101,WH-A,81.0
1,2,102,WH-A,20.0
2,3,103,WH-A,20.0
3,4,104,WH-A,20.0
4,5,105,WH-A,20.0
...,...,...,...,...
995,996,1096,WH-B,135.0
996,997,1097,WH-C,126.0
997,998,1098,WH-C,63.0
998,999,1099,WH-A,1.0


In [288]:
inv_raw.isna().sum()

InventoryID      0
ProductID        0
WarehouseCode    0
StockLevel       0
dtype: int64

In [ ]:
inv_raw.to_csv(Path.cwd().parent / "Data" / "Processed" / "investory_levels.csv")

In [290]:
prod_raw.to_csv(Path.cwd().parent / "Data" / "Processed" / "products.csv")

In [291]:
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

In [292]:
cur.execute(
    """
    CREATE TABLE IF NOT EXISTS products(
    ProductID INTEGER PRIMARY KEY, 
    ProductName TEXT,
    Category TEXT,
    Price REAL
    );
    """
)

cur.execute(
    """
    CREATE TABLE IF NOT EXISTS investory(
    InventoryID INTEGER PRIMARY KEY,
    ProductID INTEGER REFERENCES products (ProductID),
    WarehouseCode TEXT,
    StockLevel INTEGER
    );
    """
)

cur.execute(
    """
    CREATE TABLE IF NOT EXISTS sales(
    SaleID INTEGER PRIMARY KEY AUTOINCREMENT,
    ProductID INTEGER REFERENCES products (ProductID),
    QuantitySold INTEGER,
    SaleDate TEXT DEFAULT CURRENT_TIMESTAMP
    );
    """
)

conn.commit()

In [293]:
prod_raw.to_sql("products", conn, if_exists="replace", index=False)
inv_raw.to_sql("investory", conn, if_exists="replace", index=False)

1000

In [294]:
cur.execute(
    """
    CREATE TRIGGER IF NOT EXISTS UpdateStockAfterSale
    AFTER INSERT ON sales
    BEGIN
        UPDATE Inventory
        SET StockLevel = StockLevel - NEW.QuantitySold
        WHERE ProductID = NEW.ProductID;
    END;
    """
)

cur.execute("""
    CREATE VIEW IF NOT EXISTS CategoryRevenueSummary AS
    SELECT
        p.Category,
        SUM(p.Price * i.StockLevel) AS TotalPotentialRevenue
    FROM products p
    JOIN inventory i
    ON p.ProductID = i.ProductID
    GROUP BY p.Category;
    """
)

cur.execute("""
    UPDATE products
    SET Price = Price * 0.8
    WHERE ProductID IN (
        SELECT ProductID
        FROM investory
        WHERE WarehouseCode = 'WH-A'
          AND StockLevel < 40
    );
    """
)

conn.commit()
conn.close()

In [295]:
conn = sqlite3.connect(DB_PATH)
df_p = pd.read_sql("""SELECT * FROM products""", conn)
df_l = pd.read_sql("""SELECT * FROM investory""", conn)
conn.close()

In [296]:
df_p

,ProductID,ProductName,Category,Price
0,101,Product_1,Books,225.090
1,102,Product_2,Home,350.920
2,103,Product_3,Electronics,180.072
3,104,Product_4,Home,279.520
4,105,Product_5,Home,224.264
...,...,...,...,...
995,1096,Product_996,Sports,225.090
996,1097,Product_997,Toys,225.090
997,1098,Product_998,Clothing,333.560
998,1099,Product_999,Sports,367.800


In [297]:
df_l

,InventoryID,ProductID,WarehouseCode,StockLevel
0,1,101,WH-A,81.0
1,2,102,WH-A,20.0
2,3,103,WH-A,20.0
3,4,104,WH-A,20.0
4,5,105,WH-A,20.0
...,...,...,...,...
995,996,1096,WH-B,135.0
996,997,1097,WH-C,126.0
997,998,1098,WH-C,63.0
998,999,1099,WH-A,1.0
